<a href="https://colab.research.google.com/github/ashtonliu88/MidiClassifier/blob/main/MidiClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # !pip install mido==1.2.9
!pip install pretty_midi
!pip install miditok
#!pip install pydub
#!pip install pyfluidsynth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=fcaffc44ac0e6c39dafe1822c605540642bd45f8a0c30d9f1d72a6df23932e6b
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.0 MB/s eta 0:00:00


In [ ]:
#import statements
import os
from pathlib import Path
import re
import requests
import kagglehub
import pretty_midi
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from miditok import REMI
from miditok import MusicTokenizer, TokSequence
from miditok.classes import TokenizerConfig

In [ ]:
path = kagglehub.dataset_download("imsparsh/musicnet-dataset")
!mv /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1 data
path = kagglehub.dataset_download("soumikrakshit/classical-music-midi")
!mv /root/.cache/kagglehub/datasets/soumikrakshit/classical-music-midi/versions/1 data2


100%|██████████| 21.5G/21.5G [11:00<00:00, 35.0MB/s]

Extracting files...


100%|██████████| 2.33M/2.33M [00:00<00:00, 3.80MB/s]

Extracting files...


In [ ]:
#merge midi data into 1 track
def merge_instruments(midi_data):
    merged_instrument = pretty_midi.Instrument(program=0)
    for instrument in midi_data.instruments:
      if instrument.program == 0:
        merged_instrument.notes.extend(instrument.notes)
        merged_instrument.control_changes.extend(instrument.control_changes)
    merged_instrument.notes.sort(key=lambda note: note.start)
    merged_midi = pretty_midi.PrettyMIDI()
    merged_midi.instruments.append(merged_instrument)
    return merged_midi

#convert to 100 length note sequences
def vectorize_midis(midi_data, sequence_length, overlap):
  vectorized_data = []
  for midi in midi_data:
    midi_data = []
    for note in midi.instruments[0].notes:
      data = [note.end-note.start, note.pitch, note.start, note.velocity]
      midi_data.append(data)
    #vectorized_data.append(midi_data)
    start = 0
    end = sequence_length
    while(end < len(midi_data)):
      sequence = midi_data[start:end]
      vectorized_data.append(sequence)
      start, end = end - overlap, end + sequence_length - overlap
    sequence = midi_data[start:end]
    sequence.extend([[0,0,0,0]] * (sequence_length-len(sequence)))
    vectorized_data.append(sequence)
  return vectorized_data

#extract data from midi files
def get_midi_files(folder_path, sequence_length, overlap):
    midi_data = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith('.mid'):
            try:
              midi_file = pretty_midi.PrettyMIDI(file_path)
              merged_midi = merge_instruments(midi_file)
              if len(merged_midi.instruments[0].notes) > 0:
                midi_data.append(merged_midi)
            except:
              continue
    vectorized_data = vectorize_midis(midi_data, sequence_length, overlap)
    return vectorized_data


composers = ["Bach", "Beethoven", "Mozart", "chopin", "tschai"]
composers2 = ["bach", "beeth", "mozart"]

midi_data = {}
sequence_length = 200
overlap = 20

for i in range(3):
  midi_data[composers[i]] = get_midi_files(f'data/musicnet_midis/musicnet_midis/{composers[i]}', sequence_length, overlap)
  midi_data[composers[i]].extend(get_midi_files(f'data2/{composers2[i]}', sequence_length, overlap))
for i in range(3, 5):
  midi_data[composers[i]] = get_midi_files(f'data2/{composers[i]}', sequence_length, overlap)


In [ ]:
for key, value in midi_data.items():
  print(key, len(value))

Bach 197
Beethoven 2734
Mozart 463
chopin 459
tschai 111


In [ ]:
all_sequences = []
labels = []
i = 0
for value in midi_data.values():
  for sequence in value:
    all_sequences.append(sequence)
  labels.extend([i] * len(value))
  i += 1
#all_sequences = pad_sequences(all_sequences, padding='post', dtype='float32')
all_sequences = np.array(all_sequences)
labels = np.array(labels)
# Create train-test split
X_train, X_test, y_train, y_test = train_test_split(all_sequences, labels, test_size=0.2, random_state=42)

In [ ]:
#load new model
model = Sequential([
    LSTM(128, input_shape=(None, 4), return_sequences=True),
    Dropout(0.4),
    LSTM(64, return_sequences=False),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_28 (LSTM)                       │ (None, None, 128)           │          68,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_28 (Dropout)                 │ (None, None, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_29 (LSTM)                       │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_29 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 123,909 (484.02 KB)

 Trainable params: 123,909 (484.02 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#load exisiting model if you have one saved
# you need the file 'midi_classifier.keras' file to work
model = tf.keras.models.load_model('midi_classifier.keras')
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_26 (LSTM)                       │ (None, None, 128)           │          68,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, None, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_27 (LSTM)                       │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 359,249 (1.37 MB)

 Trainable params: 119,749 (467.77 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 239,500 (935.55 KB)

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)
#!mkdir classifier
#model.save_weights('./classifier/classifier.weights.h5')
model.save('midi_classifier.keras')

Epoch 1/5
80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6558 - loss: 1.1076 - val_accuracy: 0.6929 - val_loss: 0.8222
Epoch 2/5
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6977 - loss: 0.7736 - val_accuracy: 0.6866 - val_loss: 0.8026
Epoch 3/5
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.6997 - loss: 0.7597 - val_accuracy: 0.6929 - val_loss: 0.7884
Epoch 4/5
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.7291 - loss: 0.7192 - val_accuracy: 0.6976 - val_loss: 0.7847
Epoch 5/5
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7084 - loss: 0.7175 - val_accuracy: 0.7087 - val_loss: 0.7547


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7477 - loss: 0.6700
Test Accuracy: 0.72
